In [2]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import *
from pyspark.sql import Row

In [3]:
spark = SparkSession.builder.appName("spark_definitive_guide").getOrCreate()

23/04/25 21:35:33 WARN Utils: Your hostname, AS-MAC-0006.local resolves to a loopback address: 127.0.0.1; using 192.168.29.46 instead (on interface en0)
23/04/25 21:35:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/25 21:35:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.stop()

In [6]:
path = "./../src/retail-data/by-day/2010-12-01.csv"
df = spark.read.format('csv').option("header", "true").option("inferSchema", "true").load(path)
# df.printSchema()
df.createOrReplaceTempView("dfTable")

In [6]:
df.where("InvoiceNo <> 536365").selectExpr("InvoiceNo", "Description").show()
df.describe().show()

# applying multiple filters into one liner
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1 # like string contains given subString 1 or more times
newQuality = pow(col("Quantity") * col("UnitPrice"), 2) + 5 # like an expression

df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter)).where("isExpensive")\
  .withColumn("realQuantity", newQuality)\
  .show()

+---------+--------------------+
|InvoiceNo|         Description|
+---------+--------------------+
|   536366|HAND WARMER UNION...|
|   536366|HAND WARMER RED P...|
|   536367|ASSORTED COLOUR B...|
|   536367|POPPY'S PLAYHOUSE...|
|   536367|POPPY'S PLAYHOUSE...|
|   536367|FELTCRAFT PRINCES...|
|   536367|IVORY KNITTED MUG...|
|   536367|BOX OF 6 ASSORTED...|
|   536367|BOX OF VINTAGE JI...|
|   536367|BOX OF VINTAGE AL...|
|   536367|HOME BUILDING BLO...|
|   536367|LOVE BUILDING BLO...|
|   536367|RECIPE BOX WITH M...|
|   536367| DOORMAT NEW ENGLAND|
|   536368|JAM MAKING SET WI...|
|   536368|RED COAT RACK PAR...|
|   536368|YELLOW COAT RACK ...|
|   536368|BLUE COAT RACK PA...|
|   536369|BATH BUILDING BLO...|
|   536370|ALARM CLOCK BAKEL...|
+---------+--------------------+
only showing top 20 rows



+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|               -24|               0.0|           12431.0|     Australia|
|    max|          C

In [12]:
# String methods

# initcap: capitalize each word
df.select(initcap(lit("capitalize each word")), 
          lower(lit("LOWER CASE THIS")), 
          upper(lit("upper case this")),
          ltrim(lit("   trim pre-spaces")), 
          rtrim(lit("trim post-spaces   ")),
          trim(lit("  trim both sides  ")),
          lpad(lit("add 3rd arg to left"), 3, " "),
          rpad(lit("add 3rd arg to right"), 3, " ")
        ).show(1) 


+-----------------------------+----------------------+----------------------+-------------------------+--------------------------+-------------------------+-------------------------------+--------------------------------+
|initcap(capitalize each word)|lower(LOWER CASE THIS)|upper(upper case this)|ltrim(   trim pre-spaces)|rtrim(trim post-spaces   )|trim(  trim both sides  )|lpad(add 3rd arg to left, 3,  )|rpad(add 3rd arg to right, 3,  )|
+-----------------------------+----------------------+----------------------+-------------------------+--------------------------+-------------------------+-------------------------------+--------------------------------+
|         Capitalize Each Word|       lower case this|       UPPER CASE THIS|          trim pre-spaces|          trim post-spaces|          trim both sides|                            add|                             add|
+-----------------------------+----------------------+----------------------+-------------------------+---------

22/10/10 06:40:45 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 915887 ms exceeds timeout 120000 ms
22/10/10 06:40:45 WARN SparkContext: Killing executors is not supported by current scheduler.


In [9]:
# df.select("Description").show(10, False)

regex_replace_str = "BLACK|WHITE|RED|GREEN|BLUE"
regex_extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
# Replacing BLACK, WHITE, RED, GREEN, BLUE with COLOR
df.select(regexp_replace(col("Description"), regex_replace_str, "COLOR").alias("color_clean"), "Description")#.show(2, False)

# Replace character wise.. L -> 1, E -> 3, T -> 7
df.select(translate(col("Description"), "LEET", "1337"), "Description")#.show(2, False)

# extract from column if words are there in the value
df.select(regexp_extract(col("Description"), regex_extract_str, 1).alias("color_clean"), "Description")#.show(2, False)

# check if column has sub_string or not
containsBlack = instr("Description", "BLACK") >= 1
containsWhite = instr("Description", "WHITE") >= 1
df.withColumn("hasBlackOrWhite", containsBlack | containsWhite).where("hasBlackOrWhite").select("Description").show()


+--------------------+
|         Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
|RED WOOLLY HOTTIE...|
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
|WOOD 2 DRAWER CAB...|
|WOOD S/3 CABINET ...|
|WOODEN PICTURE FR...|
|WOODEN FRAME ANTI...|
|RED WOOLLY HOTTIE...|
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
|WOOD 2 DRAWER CAB...|
|WOOD S/3 CABINET ...|
|WOODEN PICTURE FR...|
|WOODEN FRAME ANTI...|
|RED WOOLLY HOTTIE...|
|JUMBO  BAG BAROQU...|
|WHITE SPOT RED CE...|
|FELT EGG COSY WHI...|
+--------------------+
only showing top 20 rows



In [34]:
df.select(monotonically_increasing_id()).show()

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
|                            5|
|                            6|
|                            7|
|                            8|
|                            9|
|                           10|
|                           11|
|                           12|
|                           13|
|                           14|
|                           15|
|                           16|
|                           17|
|                           18|
|                           19|
+-----------------------------+
only showing top 20 rows

